# COVID-19 SEIRPD-Q model

## Table of Contents

1. [Importing libs](#importing)

2. [Loading data](#loading)

3. [Data cleaning](#cleaning)

4. [(Very) Basic EDA](#eda)

5. [Epidemiology models](#models)

6. [Programming SEIRPD-Q model in Python](#implementations)

7. [Least-squares fitting](#least-squares)

8. [Extrapolation/Predictions](#deterministic-predictions)

9. [Forward UQ](#uq)

10. [Bayesian Calibration](#bayes-calibration)

Before analyze the models, we begin having a look at the available data.

<a id="importing"></a>
## Importing libs

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.integrate import solve_ivp # to solve ODE system
from scipy import optimize # to solve minimization problem from least-squares fitting
from numba import jit # to accelerate ODE system RHS evaluations
import pymc3 as pm # for uncertainty quantification and model calibration
import theano # to control better pymc3 backend and write a wrapper
import theano.tensor as t # for the wrapper to a custom model to pymc3

# Plotting libs
import matplotlib.pyplot as plt
import altair as alt

seed = 12345 # for the sake of reproducibility :)
np.random.seed(seed)

plt.style.use('seaborn-talk') # beautify the plots!

THEANO_FLAGS='optimizer=fast_compile' # A theano trick

<a id="loading"></a>
## Loading data

In [ ]:
df_covid = pd.read_csv("../pydemic/data/covid_19_clean_complete.csv", parse_dates=['Date'])

df_covid.info()

In [ ]:
df_covid.head()

In [ ]:
columns_to_filter_cases = ["Country/Region", "Date", "Confirmed", "Deaths"]
df_covid_cases = df_covid[columns_to_filter_cases]

df_covid_cases.head()

<a id="cleaning"></a>
## Data cleaning

Let's do a data cleaning based in this [amazing notebook!](https://www.kaggle.com/abhinand05/covid-19-digging-a-bit-deeper)

In [ ]:
print(f"First day entry:\t {df_covid['Date'].min()}")
print(f"Last day reported:\t {df_covid['Date'].max()}")
print(f"Total of tracked days:\t {df_covid['Date'].max() - df_covid['Date'].min()}")

In [ ]:
df_covid.rename(
    columns={
        'Date': 'date', 
        'Province/State':'state',
        'Country/Region':'country',
        'Last Update':'last_updated',
        'Confirmed': 'confirmed',
        'Deaths':'deaths',
        'Recovered':'recovered'}, 
    inplace=True
)

df_covid

Active Case = confirmed - deaths - recovered

In [ ]:
df_covid['active'] = df_covid['confirmed'] - df_covid['deaths'] - df_covid['recovered']

df_covid

Replacing Mainland china with just China:

In [ ]:
df_covid['country'] = df_covid['country'].replace('Mainland China', 'China')

df_covid

<a id="eda"></a>
## (Very) Basic EDA

Worldwide scenario:

In [ ]:
df_grouped = df_covid.groupby('date')['date', 'confirmed', 'deaths'].sum().reset_index()

df_grouped

In [ ]:
confirmed_plot = alt.Chart(df_grouped).mark_circle(size=60, color='blue').encode(
    x=alt.X('date', axis=alt.Axis(title='Date')),
    y=alt.Y('confirmed', axis=alt.Axis(title='Cases'))
)

deaths_plot = alt.Chart(df_grouped).mark_circle(size=60, color='red').encode(
    x='date',
    y='deaths'
)

worldwide_plot = confirmed_plot + deaths_plot
worldwide_plot.interactive()

Now, let's take a look at Brazil:

In [ ]:
def get_df_country_cases(df: pd.DataFrame, country_name: str) -> pd.DataFrame:
    df_grouped_country = df[df['country'] == country_name].reset_index()
    df_grouped_country_date = df_grouped_country.groupby('date')['date', 'confirmed', 'deaths', 'recovered'].sum().reset_index()
    df_grouped_country_date["confirmed_marker"] = df_grouped_country_date.shape[0] * ['Confirmed']
    df_grouped_country_date["deaths_marker"] = df_grouped_country_date.shape[0] * ['Deaths']
    return df_grouped_country_date


def get_df_state_cases(df: pd.DataFrame, state_name: str) -> pd.DataFrame:
    df_grouped_state = df[df['state'] == state_name].reset_index()
    df_grouped_state_date = df_grouped_state.groupby('date')['date', 'confirmed', 'deaths', 'recovered'].sum().reset_index()
    df_grouped_state_date["confirmed_marker"] = df_grouped_state_date.shape[0] * ['Confirmed']
    df_grouped_state_date["deaths_marker"] = df_grouped_state_date.shape[0] * ['Deaths']
    return df_grouped_state_date

In [ ]:
def altair_plot_for_confirmed_and_deaths(df_grouped: pd.DataFrame, data_at_x_axis: str='date') -> alt.Chart:
    confirmed_plot = alt.Chart(df_grouped).mark_circle(size=60).encode(
        x=alt.X(data_at_x_axis, axis=alt.Axis(title='Date')),
        y=alt.Y('confirmed', axis=alt.Axis(title='Cases'), title='Confirmed'),
        color=alt.Color("confirmed_marker", title="Cases"),
    )

    deaths_plot = alt.Chart(df_grouped).mark_circle(size=60).encode(
        x=data_at_x_axis,
        y='deaths',
        color=alt.Color("deaths_marker"),
    )

    return confirmed_plot + deaths_plot

In [ ]:
df_grouped_brazil = get_df_country_cases(df_covid, "Brazil")

df_grouped_brazil

In [ ]:
altair_plot_for_confirmed_and_deaths(df_grouped_brazil).interactive()

Let's have a look at China:

In [ ]:
df_grouped_china = get_df_country_cases(df_covid, "China")

df_grouped_china

In [ ]:
altair_plot_for_confirmed_and_deaths(df_grouped_china).interactive()

Now, let's take a look only at Hubei, which is disease focus in China:

In [ ]:
df_grouped_hubei = get_df_state_cases(df_covid, "Hubei")

df_grouped_hubei[:30]

In [ ]:
altair_plot_for_confirmed_and_deaths(df_grouped_hubei).interactive()

Now a look at Italy:

In [ ]:
df_grouped_italy = get_df_country_cases(df_covid, "Italy")

df_grouped_italy

In [ ]:
altair_plot_for_confirmed_and_deaths(df_grouped_italy).interactive()

### Comparison between Brazil and Italy

In [ ]:
df_brazil_cases_by_day = df_grouped_brazil[df_grouped_brazil.confirmed > 0]
df_brazil_cases_by_day = df_brazil_cases_by_day.reset_index(drop=True)
df_brazil_cases_by_day['day'] = df_brazil_cases_by_day.date.apply(lambda x: (x - df_brazil_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_brazil_cases_by_day = df_brazil_cases_by_day[reordered_columns]

df_brazil_cases_by_day

In [ ]:
df_italy_cases_by_day = df_grouped_italy[df_grouped_italy.confirmed > 0]
df_italy_cases_by_day = df_italy_cases_by_day.reset_index(drop=True)
df_italy_cases_by_day['day'] = df_italy_cases_by_day.date.apply(lambda x: (x - df_italy_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_italy_cases_by_day = df_italy_cases_by_day[reordered_columns]

df_italy_cases_by_day

In [ ]:
df_italy_cases_by_day_limited_by_br = df_italy_cases_by_day[df_italy_cases_by_day.day <= df_brazil_cases_by_day.day.max()]
days = df_brazil_cases_by_day.day

plt.figure(figsize=(9, 6))
plt.plot(days, df_brazil_cases_by_day.confirmed, marker='X', linestyle="", markersize=10, label='Confirmed (BR)')
plt.plot(days, df_italy_cases_by_day_limited_by_br.confirmed, marker='o', linestyle="", markersize=10, label='Confirmed (ITA)')
plt.plot(days, df_brazil_cases_by_day.deaths, marker='s', linestyle="", markersize=10, label='Deaths (BR)')
plt.plot(days, df_italy_cases_by_day_limited_by_br.deaths, marker='*', linestyle="", markersize=10, label='Deaths (ITA)')

plt.xlabel("Day(s)")
plt.ylabel("Cases")
plt.legend()
plt.grid()

plt.show()

### China scenario since first entry

In [ ]:
df_china_cases_by_day = df_grouped_china[df_grouped_china.confirmed > 0]
df_china_cases_by_day = df_china_cases_by_day.reset_index(drop=True)
df_china_cases_by_day['day'] = df_china_cases_by_day.date.apply(lambda x: (x - df_china_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_china_cases_by_day = df_china_cases_by_day[reordered_columns]

df_china_cases_by_day

And for Hubei:

In [ ]:
df_hubei_cases_by_day = df_grouped_hubei[df_grouped_hubei.confirmed > 0]
df_hubei_cases_by_day = df_hubei_cases_by_day.reset_index(drop=True)
df_hubei_cases_by_day['day'] = df_hubei_cases_by_day.date.apply(lambda x: (x - df_hubei_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_hubei_cases_by_day = df_hubei_cases_by_day[reordered_columns]

df_hubei_cases_by_day

### Spain since first recorded case

In [ ]:
df_grouped_spain = get_df_country_cases(df_covid, "Spain")
df_spain_cases_by_day = df_grouped_spain[df_grouped_spain.confirmed > 0]
df_spain_cases_by_day = df_spain_cases_by_day.reset_index(drop=True)
df_spain_cases_by_day['day'] = df_spain_cases_by_day.date.apply(lambda x: (x - df_spain_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_spain_cases_by_day = df_spain_cases_by_day[reordered_columns]

df_spain_cases_by_day

### Iran since first case

In [ ]:
df_grouped_iran = get_df_country_cases(df_covid, "Iran")
df_iran_cases_by_day = df_grouped_iran[df_grouped_iran.confirmed > 0]
df_iran_cases_by_day = df_iran_cases_by_day.reset_index(drop=True)
df_iran_cases_by_day['day'] = df_iran_cases_by_day.date.apply(lambda x: (x - df_iran_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_iran_cases_by_day = df_iran_cases_by_day[reordered_columns]

df_iran_cases_by_day

### USA since first case

In [ ]:
df_grouped_usa = get_df_country_cases(df_covid, "US")
df_usa_cases_by_day = df_grouped_usa[df_grouped_usa.confirmed > 0]
df_usa_cases_by_day = df_usa_cases_by_day.reset_index(drop=True)
df_usa_cases_by_day['day'] = df_usa_cases_by_day.date.apply(lambda x: (x - df_usa_cases_by_day.date.min()).days)

reordered_columns = ['date', 'day', 'confirmed', 'deaths', 'recovered', 'confirmed_marker', 'deaths_marker']
df_usa_cases_by_day = df_usa_cases_by_day[reordered_columns]

df_usa_cases_by_day

<a id="models"></a>
## Epidemiology models

Now, let me explore the data in order to calibrate an epidemiologic model in order to try to simulate and predict cases.

### Classical models

Here I present a brief review of classical temporal models (space dependency is not considered). Then I proposed modifications for such models.

#### SIR model

The model represents an epidemic scenario, aiming to predict and control infectious diseases. It consists in a non-linear dynamical system, which considers populational sub-groups according to the state of the individuals. A simple model would be composed by 3 subgroups:

* Susceptible individuals (S);
* Infected (I);
* Recovered (R).

With such components, a classical dynamical system known as SIR model. The equations of such a system is written as:

\begin{align*}
  \dot{S} &= - \beta S I \\ 
  \dot{I} &= \beta S I - \zeta I \\ 
  \dot{R} &= \zeta I
\end{align*}

where $\dot{(\bullet)}$ stands for time-derivative.

Some biological explanation for parameters:

* $\beta$ is the conversion parameter due to interaction between a susceptible individual with an infected one;
* $\zeta$ is the conversion parameter related to the recovery rate. In other words, the individuals that become immune;

#### SEIR model

Another classical model known as SEIR (Susceptible-Exposed-Infected-Recovered) is common applied in Computational Epidemiology literature (you can check it elsewhere). In this model, a new sub-group of individuals is considered: Exposed. Such individuals are those that are infected, but don't show any sympton. In the classical SEIR model, exposed individuals **do not transmit the disease**. The ODE system now becomes:

\begin{align*}
    \dot{S} &= - \beta S  I \\
    \dot{E} &= \beta S I - \alpha E \\
    \dot{I} &= \alpha E - \zeta I \\
    \dot{R} &= \zeta I \\
\end{align*}

Brief biological interpretation for additional parameter:

* $\alpha$ is the conversion parameter for exposed individuals that transformed into infected ones.

### Modified models

Here, I propose some simple modifications in order to improve model representability for COVID-19.

#### Modified SIR model (SIRD)

In this model, deaths due to the disease is considered explicitly. A new individuals sub-group is introduced: dead individuals. To consider such phenomenon, an additional equation is required, as well as a modification in the Infected equation balance. The ODE system is given below:

\begin{align*}
  \dot{S} &= - \beta S I \\ 
  \dot{I} &= \beta S I - \zeta I - \delta I \\ 
  \dot{R} &= \zeta I \\
  \dot{D} &= \delta I
\end{align*}

Brief biological interpretation for additional parameter:

* $\delta$ is the mortality rate for the disease.

#### Modified SEIR model (SEIR-2)

This model aims to solve the lack of the original SEIR model, which does not consider disease transmission between exposed and susceptible individuals. In order to take it into account,
we modified balance equations for S and E as follows:

\begin{align*}
    \dot{S} &= - \beta S  I  - \gamma S E \\
    \dot{E} &= \beta S I - \alpha E + \gamma S E \\
    \dot{I} &= \alpha E - \zeta I \\
    \dot{R} &= \zeta I \\
\end{align*}

Brief biological interpretation for additional parameter:

* $\gamma$ is the conversion rate parameter for susceptible individuals that interact with exposed individuals and then become exposed.

#### Modified SEIR model with deaths (SEIRD)

Very similiar to the last one, but it considers a sub-population of dead individuals due to the disease. Thus, the model is written as:

\begin{align*}
    \dot{S} &= - \beta S  I  - \gamma S E \\
    \dot{E} &= \beta S I - \alpha E + \gamma S E \\
    \dot{I} &= \alpha E - \zeta I - \delta I \\
    \dot{R} &= \zeta I \\
    \dot{D} &= \delta I
\end{align*}

#### Modified SEIRD model considering quarantine lockdown (SEIRD-Q)

This is a modified model that take into account a removal rate from Susceptible, Exposed and Infected individuals to quarantine. The main hypothesis is that this conversion
is under a constant removal parameter (by time, i.e., 1 / time), and after the conversion, the individual becomes "Recovered" and can not transmit the disease anymore. The new system is written as

\begin{align*}
    \dot{S} &= - \beta S I  - \gamma S E - \omega S\\
    \dot{E} &= \beta S I - \alpha E + \gamma S E - \omega E \\
    \dot{I} &= \alpha E - \zeta I - \delta I - \omega I \\
    \dot{R} &= \zeta I + \omega (S + E + I) \\
    \dot{D} &= \delta I
\end{align*}

Brief biological interpretation for additional parameter:

* $\omega$ is the conversion rate parameter for Susceptible, Exposed and Infected individuals that becomes Recovered due to a removal to a quarantine.

### Remarks for the models units

All sub-population variables (S, I, R, etc) are dimensionless. To obtain the variables, we have to consider that

\begin{align*}
    &S := \frac{\mathcal{S}}{N} \\
    &E := \frac{\mathcal{E}}{N} \\
    &I := \frac{\mathcal{I}}{N} \\
    &R := \frac{\mathcal{R}}{N} \\
    &D := \frac{\mathcal{D}}{N} \\
\end{align*}

with $N$ denoting the total population and $\mathcal{S}$, $\mathcal{E}$, $\mathcal{I}$, $\mathcal{R}$ and $\mathcal{D}$ as the absolute sub-population amounts. Therefore, S, E, I, R and D are given as fractions of the total population.

<a id="implementations"></a>
## Programming SEIRPD-Q model in Python

In [ ]:
@jit(nopython=True)
def seirpdq_model(t, X, alpha=1/5, beta=1, gamma=0, omega=0, zeta=1/15, delta=0.02, theta=0.01, eta=1e-5, chi=0.4):
    """
    SEIRPD-Q python implementation.
    """
    S, E, I, R, P, D = X
    S_prime = - beta * S * I - gamma * E * S - omega * S + eta * R
    E_prime = beta * S * I - alpha * E + gamma * E * S - omega * E
    I_prime = alpha * E - zeta * I - delta * I - omega * I - chi * I
    R_prime = zeta * I + (1 - theta) * P + omega * (S + E + I) - eta * R
    P_prime = chi * I - zeta * P - theta * P - (1 - theta) * P
    D_prime = delta * I + theta * P
    return S_prime, E_prime, I_prime, R_prime, P_prime, D_prime

ODE solvers wrapper using `scipy.integrate.solve_ivp`:

In [ ]:
def seirpdq_ode_solver(y0, t_span, t_eval, beta=1, gamma=0, delta=0.02, theta=0.01, omega=1e-2, alpha=1/4, zeta=1/14, eta=1e-5, chi=0.7):
    solution_ODE = solve_ivp(
        fun=lambda t, y: seirpdq_model(t, y, alpha=alpha, beta=beta, gamma=gamma, omega=omega, zeta=zeta, delta=delta, eta=eta, chi=chi), 
        t_span=t_span, 
        y0=y0,
        t_eval=t_eval,
        method='LSODA'
    )
    
    return solution_ODE

Getting population for each country:

In [ ]:
df_population = pd.read_csv("../pydemic/data/countries of the world.csv")

df_population

In [ ]:
brazil_population = float(df_population[df_population.Country == 'Brazil '].Population)
italy_population = float(df_population[df_population.Country == 'Italy '].Population)
china_population = float(df_population[df_population.Country == 'China '].Population)
hubei_population = float(58500000)  # from wikipedia!
spain_population = float(df_population[df_population.Country == 'Spain '].Population)
iran_population = float(df_population[df_population.Country == 'Iran '].Population)
us_population = float(df_population[df_population.Country == 'United States '].Population)

target_population = italy_population
target_population

Initial Conditions:

In [ ]:
df_target_country = df_italy_cases_by_day
E0, I0, R0, P0, D0 = 5 * float(df_target_country.confirmed[0]), 1.2 * float(df_target_country.confirmed[0]), float(df_target_country.recovered[0]), float(df_target_country.confirmed[0]), float(df_target_country.deaths[0])
S0 = target_population - (E0 + I0 + R0 + P0 + D0)
y0_seirpdq = S0 / target_population, E0 / target_population, I0 / target_population, R0 / target_population, P0 / target_population, D0 / target_population  # SEIRPDQ IC array
print(y0_seirpdq)

<a id="least-squares"></a>
## Least-Squares fitting

Now, we can know how to solve the forward problem, so we can try to fit it with a non-linear Least-Squares method for parameter estimation. Let's begin with a generic Least-Square formulation:

In [ ]:
def seirpdq_least_squares_error_ode(par, time_exp, f_exp, fitting_model, initial_conditions):
    args = par
    f_exp1, f_exp2 = f_exp
    time_span = (time_exp.min(), time_exp.max())
    
    y_model = fitting_model(initial_conditions, time_span, time_exp, *args)
    simulated_time = y_model.t
    simulated_ode_solution = y_model.y
    _, _, _, _, simulated_qoi1, simulated_qoi2 = simulated_ode_solution
    
    residual1 = f_exp1 - simulated_qoi1
    residual2 = f_exp2 - simulated_qoi2

    weighting_for_exp1_constraints = 1e0
    weighting_for_exp2_constraints = 1e0
    return weighting_for_exp1_constraints * np.sum(residual1 ** 2.0) + weighting_for_exp2_constraints * np.sum(residual2 ** 2.0)


def callback_de(xk, convergence):
    print(f'parameters = {xk}\n')

Setting fitting domain (given time for each observation) and the observations (observed population at given time):

In [ ]:
data_time = df_target_country.day.values.astype(np.float64)
infected_individuals = df_target_country.confirmed.values / target_population
dead_individuals = df_target_country.deaths.values / target_population

To calibrate the model, we define an objective function, which is a Least-Squares function in the present case, and minimize it. To (*try to*) avoid local minima, we use Differential Evolution (DE) method (see this [nice presentation](https://www.maths.uq.edu.au/MASCOS/Multi-Agent04/Fleetwood.pdf) to get yourself introduced to this great subject). In summary, DE is a family of Evolutionary Algorithms that aims to solve Global Optimization problems. Moreover, DE is derivative-free and population-based method.

Below, calibration is performed for selected models:

In [ ]:
num_of_parameters_to_fit_seirpdq = 4
bounds_seirpdq = num_of_parameters_to_fit_seirpdq * [(0, 0.5)]
# bounds_seirdaq = [(0, 1e-2), (0, 1), (0, 1), (0, 0.2), (0, 0.2), (0, 0.2)]

result_seirpdq = optimize.differential_evolution(
    seirpdq_least_squares_error_ode, 
    bounds=bounds_seirpdq, 
    args=(data_time, [infected_individuals, dead_individuals], seirpdq_ode_solver, y0_seirpdq), 
    popsize=30,
    strategy='best1bin',
    tol=1e-2,
    recombination=0.7,
#         mutation=0.7,
    maxiter=500,
    polish=True,
    disp=True,
    seed=seed,
    callback=callback_de,
    workers=-1
)

print(result_seirpdq)

In [ ]:
# beta_fitted_seirdaq, gamma_I_fitted_seirdaq = result_seirdaq.x  # SEIRDAQ parameters
# beta_fitted_seirpdq, gamma_fitted_seirpdq, delta_fitted_seirpdq, omega_fitted_seirpdq = result_seirpdq.x  # SEIRDAQ parameters
# beta_fitted_seirpdq, gamma_fitted_seirpdq, delta_fitted_seirpdq = result_seirpdq.x  # SEIRDAQ parameters

In [ ]:
t0 = data_time.min()
tf = data_time.max()

solution_ODE_seirpdq = seirpdq_ode_solver(
    y0_seirpdq, 
    (t0, tf), 
    data_time, 
    *result_seirpdq.x
)
t_computed_seirpdq, y_computed_seirpdq = solution_ODE_seirpdq.t, solution_ODE_seirpdq.y
S_seirpdq, E_seirpdq, I_seirpdq, R_seirpdq, P_seirpdq, D_seirpdq = y_computed_seirpdq

In [ ]:
# model_list = list()
# alpha_list = list()
# beta_list = list()
# delta_list = list()
# gamma_list = list()
# omega_list = list()
# zeta_list = list()

# if has_to_run_sir:
#     model_list.append("SIR")
#     alpha_list.append("-")
#     beta_list.append(np.float(beta_fitted_sir))
#     delta_list.append("-")
#     gamma_list.append("-")
#     omega_list.append("-")
#     zeta_list.append(zeta_fitted)

# if has_to_run_sird:
#     model_list.append("SIRD")
#     alpha_list.append("-")
#     beta_list.append(beta_fitted_sird)
#     delta_list.append(delta_fitted_sird)
#     gamma_list.append("-")
#     omega_list.append("-")
#     zeta_list.append(zeta_fitted)
    
# if has_to_run_seir:
#     model_list.append("SEIR")
#     alpha_list.append(alpha_fitted)
#     beta_list.append(beta_fitted_seir)
#     delta_list.append("-")
#     gamma_list.append(gamma_fitted_seir)
#     omega_list.append("-")
#     zeta_list.append(zeta_fitted)

# if has_to_run_seird:
#     model_list.append("SEIRD")
#     alpha_list.append(alpha_fitted)
#     beta_list.append(beta_fitted_seird)
#     delta_list.append(delta_fitted_seird)
#     gamma_list.append(gamma_fitted_seird)
#     omega_list.append("-")
#     zeta_list.append(zeta_fitted)

# if has_to_run_seirdq:
#     model_list.append("SEIRD-Q")
#     alpha_list.append(alpha_fitted)
#     beta_list.append(beta_fitted_seirdq)
#     delta_list.append(delta_fitted_seirdq)
#     gamma_list.append(gamma_fitted_seirdq)
#     omega_list.append(omega_fitted_seirdq)
#     zeta_list.append(zeta_fitted)
    
# parameters_dict = {
#     "Model": model_list,
#     r"$\alpha$": alpha_list,
#     r"$\beta$": beta_list,
#     r"$\delta$": delta_list,
#     r"$\gamma$": gamma_list,
#     r"$\omega$": omega_list,
#     r"$\zeta$": zeta_list,
# }

# df_parameters_calibrated = pd.DataFrame(parameters_dict)

# df_parameters_calibrated

In [ ]:
# print(df_parameters_calibrated.to_latex(index=False))

Show calibration result based on available data:

In [ ]:
plt.figure(figsize=(9,7))

# plt.plot(t_computed_seirdaq, I_seirdaq * target_population, label='Infected (SEIRDAQ)', marker='X', linestyle="-", markersize=10)
# plt.plot(t_computed_seirdq, R_seirdq * target_population, label='Recovered (SEIRDAQ)', marker='o', linestyle="-", markersize=10)
plt.plot(t_computed_seirpdq, P_seirpdq * target_population, label='Diagnosed (SEIRPD-Q)', marker='s', linestyle="-", markersize=10)
plt.plot(t_computed_seirpdq, D_seirpdq * target_population, label='Deaths (SEIRPD-Q)', marker='s', linestyle="-", markersize=10)
    
plt.plot(data_time, infected_individuals * target_population, label='Observed infected', marker='s', linestyle="", markersize=10)
plt.plot(data_time, dead_individuals * target_population, label='Recorded deaths', marker='v', linestyle="", markersize=10)
plt.legend()
plt.grid()
plt.xlabel('Time (days)')
plt.ylabel('Population')

plt.tight_layout()
plt.savefig("seirpdq_deterministic_calibration.png")
plt.show()

In [ ]:
methods_list = list()
deaths_list = list()

methods_list.append("SEIRPD-Q")
deaths_list.append(int(target_population * D_seirpdq.max()))
print(f"Confirmed cases estimate for today (SEIRPD-Q):\t{int(P_seirpdq.max() * target_population)}")
print(f"Confirmed cases estimate population percentage for today (SEIRPD-Q):\t{100 * P_seirpdq.max():.3f}%")
print(f"Death estimate for today (SEIRPD-Q):\t{int(D_seirpdq.max() * target_population)}")
print(f"Death estimate population percentage for today (SEIRPD-Q):\t{100 * D_seirpdq.max():.3f}%")

methods_list.append("Recorded")
deaths_list.append(int(dead_individuals[-1] * target_population))

death_estimates_dict = {"Method": methods_list, "Deaths estimate": deaths_list}
df_deaths_estimates = pd.DataFrame(death_estimates_dict)
print(f"Recorded deaths until today:\t{int(dead_individuals[-1] * target_population)}")

In [ ]:
# df_deaths_estimates.set_index("Model", inplace=True)
print(df_deaths_estimates.to_latex(index=False))

<a id="deterministic-predictions"></a>
## Extrapolation/Predictions

Now, let's extrapolate to next days.

In [ ]:
t0 = float(data_time.min())
number_of_days_after_last_record = 90
tf = data_time.max() + number_of_days_after_last_record
time_range = np.linspace(0., tf, int(tf))

solution_ODE_predict_seirpdq = seirpdq_ode_solver(
    y0_seirpdq, 
    (t0, tf), 
    time_range, 
    *result_seirpdq.x
)  # SEIRDAQ
#     solution_ODE_predict_seirdaq = seirdaq_ode_solver(y0_seirdaq, (t0, tf), time_range)  # SEIRDAQ
t_computed_predict_seirpdq, y_computed_predict_seirpdq = solution_ODE_predict_seirpdq.t, solution_ODE_predict_seirpdq.y
S_predict_seirpdq, E_predict_seirpdq, I_predict_seirpdq, R_predict_seirpdq, P_predict_seirpdq, D_predict_seirpdq = y_computed_predict_seirpdq

Calculating the day when the number of infected individuals is max:

In [ ]:
has_to_plot_infection_peak = True

crisis_day_seirpdq = np.argmax(P_predict_seirpdq) + 1

In [ ]:
plt.figure(figsize=(9,7))

#     plt.plot(t_computed_predict_seirdaq, 100 * S_predict_seirdq, label='Susceptible (SEIRD-Q)', marker='s', linestyle="-", markersize=10)
plt.plot(t_computed_predict_seirpdq, 100 * E_predict_seirpdq, label='Exposed (SEIRPD-Q)', marker='*', linestyle="-", markersize=10)
plt.plot(t_computed_predict_seirpdq, 100 * I_predict_seirpdq, label='Infected (SEIRPD-Q)', marker='X', linestyle="-", markersize=10)
#     plt.plot(t_computed_predict_seirdaq, 100 * R_predict_seirdaq, label='Recovered (SEIRDAQ)', marker='o', linestyle="-", markersize=10)
plt.plot(t_computed_predict_seirpdq, 100 * D_predict_seirpdq, label='Deaths (SEIRPD-Q)', marker='v', linestyle="-", markersize=10)
plt.plot(t_computed_predict_seirpdq, 100 * P_predict_seirpdq, label='Diagnosed (SEIRPD-Q)', marker='D', linestyle="-", markersize=10)
# plt.plot(t_computed_predict_seirdaq, Q_predict_seirdaq, label='Quarantine (SEIRDAQ)', marker='H', linestyle="-", markersize=10)
if has_to_plot_infection_peak:
    plt.axvline(x=crisis_day_seirpdq, color="red", linestyle="-", label="Diagnosed peak (SEIRPD-Q)")

plt.xlabel('Time (days)')
plt.ylabel('Population %')
plt.legend()
plt.grid()

plt.tight_layout()
plt.savefig("seirpdq_deterministic_predictions.png")
plt.show()

In [ ]:
print(f"Max number of diagnosed individuals (SEIRPD-Q model):\t {int(np.max(P_predict_seirpdq * target_population))}")
print(f"Population percentage of max number of diagnosed individuals (SEIRPD-Q model):\t {100 * np.max(P_predict_seirpdq):.2f}%")
print(f"Day estimate for max number of diagnosed individuals (SEIRPD-Q model):\t {crisis_day_seirpdq}")
print(f"Percentage of number of death estimate (SEIRPD-Q model):\t {100 * D_predict_seirpdq[-1]:.3f}%")
print(f"Number of death estimate (SEIRPD-Q model):\t {D_predict_seirpdq[-1] * target_population:.3f}")

<a id="uq"></a>
## Forward Uncertainty Propagation

In [ ]:
@theano.compile.ops.as_op(itypes=[t.dvector, t.dvector, t.dscalar, t.dscalar, t.dscalar, t.dscalar], otypes=[t.dmatrix])
def seirpdq_ode_wrapper(time_exp, initial_conditions, beta, gamma, delta, theta):
    time_span = (time_exp.min(), time_exp.max())
    
    args = [beta, gamma, delta, theta]
    y_model = seirpdq_ode_solver(initial_conditions, time_span, time_exp, *args)
    simulated_time = y_model.t
    simulated_ode_solution = y_model.y
    
    return simulated_ode_solution


# @theano.compile.ops.as_op(itypes=[t.dvector, t.dvector, t.dscalar, t.dscalar, t.dscalar], otypes=[t.dmatrix])
# def seirpdq_ode_wrapper(time_exp, initial_conditions, beta, gamma, delta):
#     time_span = (time_exp.min(), time_exp.max())
    
#     args = [beta, gamma, delta]
#     y_model = seirpdq_ode_solver(initial_conditions, time_span, time_exp, *args)
#     simulated_time = y_model.t
#     simulated_ode_solution = y_model.y
    
#     return simulated_ode_solution

In [ ]:
# beta_deterministic, gamma_deterministic, delta_deterministic, omega_deterministic = result_seirpdq.x
beta_deterministic, gamma_deterministic, delta_deterministic, theta_deterministic = result_seirpdq.x

In [ ]:
percent_uq = 5 / 100
with pm.Model() as model_mcmc:
    # Prior distributions for the model's parameters
    beta = pm.Uniform('beta', lower=(1 - percent_uq) * beta_deterministic, upper=(1 + percent_uq) * beta_deterministic)
    gamma = pm.Uniform('gamma', lower=(1 - percent_uq) * gamma_deterministic, upper=(1 + percent_uq) * gamma_deterministic)
    delta = pm.Uniform('delta', lower=(1 - percent_uq) * delta_deterministic, upper=(1 + percent_uq) * delta_deterministic)
    theta = pm.Uniform('theta', lower=(1 - percent_uq) * theta_deterministic, upper=(1 + percent_uq) * theta_deterministic)

    # Defining the deterministic formulation of the problem
    fitting_model = pm.Deterministic('seirpdq_model', seirpdq_ode_wrapper(
        theano.shared(time_range), 
        theano.shared(np.array(y0_seirpdq)),
        beta,
        gamma,
        delta,
        theta
        )
    )

    # The Monte Carlo procedure driver
    step = pm.step_methods.Metropolis()
    seirdpq_trace = pm.sample(4000, chains=4, cores=4, step=step, random_seed=seed)

In [ ]:
percentile_cut = 5

y_min = 100 * np.percentile(seirdpq_trace['seirpdq_model'], percentile_cut, axis=0)
y_max = 100 * np.percentile(seirdpq_trace['seirpdq_model'], 100 - percentile_cut, axis=0)
y_fit = 100 * np.percentile(seirdpq_trace['seirpdq_model'], 50, axis=0)

In [ ]:
plt.figure(figsize=(9, 7))

# plt.plot(time_range, 100 * y_fit[0], 'b', label='Suscetible')
# plt.fill_between(time_range, y_min[0], y_max[0], color='b', alpha=0.2)

# plt.plot(time_range, 100 * y_fit[1], 'r', label='Exposed')
# plt.fill_between(time_range, y_min[1], y_max[1], color='r', alpha=0.2)

# plt.plot(time_range, y_fit[2], 'g', label='Infected')
# plt.fill_between(time_range, y_min[2], y_max[2], color='g', alpha=0.2)

# plt.plot(data_time, 100 * y_fit[3], 'y', label='Recovered')
# plt.fill_between(data_time, y_min[3], y_max[3], color='y', alpha=0.2)

plt.plot(time_range, y_fit[4], 'b', label='Diagnosed (SEIRPD-Q)', marker='D', linestyle="-", markersize=10)
plt.fill_between(time_range, y_min[4], y_max[4], color='b', alpha=0.2)

plt.plot(time_range, y_fit[5], 'r', label='Deaths (SEIRPD-Q)', marker='v', linestyle="-", markersize=10)
plt.fill_between(time_range, y_min[5], y_max[5], color='r', alpha=0.2)

plt.xlabel('Time (days)')
plt.ylabel('Population %')
plt.legend()
plt.grid()

plt.tight_layout()

plt.savefig('seirpdq_uq.png')
plt.show()

<a id="bayes-calibration"></a>
## Bayesian Calibration

In [ ]:
observations_to_fit = np.vstack([infected_individuals, dead_individuals])

In [ ]:
@theano.compile.ops.as_op(itypes=[t.dvector, t.dvector, t.dscalar, t.dscalar, t.dscalar, t.dscalar], otypes=[t.dmatrix])
def seirpdq_ode_wrapper(time_exp, initial_conditions, beta, gamma, delta, theta):
    time_span = (time_exp.min(), time_exp.max())
    
    args = [beta, gamma, delta, theta]
    y_model = seirpdq_ode_solver(initial_conditions, time_span, time_exp, *args)
    simulated_time = y_model.t
    simulated_ode_solution = y_model.y
    _, _, _, _, simulated_qoi1, simulated_qoi2 = simulated_ode_solution
    
    concatenate_simulated_qoi = np.vstack([simulated_qoi1, simulated_qoi2])
    
    return concatenate_simulated_qoi


# @theano.compile.ops.as_op(itypes=[t.dvector, t.dvector, t.dscalar, t.dscalar, t.dscalar], otypes=[t.dmatrix])
# def seirpdq_ode_wrapper(time_exp, initial_conditions, beta, gamma, delta):
#     time_span = (time_exp.min(), time_exp.max())
    
#     args = [beta, gamma, delta]
#     y_model = seirpdq_ode_solver(initial_conditions, time_span, time_exp, *args)
#     simulated_time = y_model.t
#     simulated_ode_solution = y_model.y
#     _, _, _, _, simulated_qoi1, simulated_qoi2 = simulated_ode_solution
    
#     concatenate_simulated_qoi = np.vstack([simulated_qoi1, simulated_qoi2])
    
#     return concatenate_simulated_qoi

In [ ]:
population_uncertain_variance = 0.05 * np.max(dead_individuals) * target_population
variance = (population_uncertain_variance / target_population) * (population_uncertain_variance / target_population)
standard_deviation = np.sqrt(variance)

standard_deviation

In [ ]:
percent_calibration = 0.2
with pm.Model() as model_mcmc:
    # Prior distributions for the model's parameters
    beta = pm.Uniform('beta', lower=(1 - percent_calibration) * beta_deterministic, upper=(1 + percent_calibration) * beta_deterministic)
    gamma = pm.Uniform('gamma', lower=(1 - percent_calibration) * gamma_deterministic, upper=(1 + percent_calibration) * gamma_deterministic)
    delta = pm.Uniform('delta', lower=(1 - percent_calibration) * delta_deterministic, upper=(1 + percent_calibration) * delta_deterministic)
    theta = pm.Uniform('theta', lower=(1 - percent_uq) * theta_deterministic, upper=(1 + percent_uq) * theta_deterministic)

    # Defining the deterministic formulation of the problem
    fitting_model = pm.Deterministic('seirpdq_model', seirpdq_ode_wrapper(
        theano.shared(data_time), 
        theano.shared(np.array(y0_seirpdq)),
        beta,
        gamma,
        delta,
        theta
        )
    )
    
    likelihood_model = pm.Normal('likelihood_model', mu=fitting_model, sigma=standard_deviation, observed=observations_to_fit)

    # The Monte Carlo procedure driver
    step = pm.step_methods.Metropolis()
    seirdpq_trace_calibration = pm.sample(50000, chains=4, cores=4, step=step, random_seed=seed, tune=10000)

In [ ]:
plot_step = 100

In [ ]:
pm.traceplot(seirdpq_trace_calibration[::plot_step], var_names=('beta'))
plt.savefig('seirpdq_beta_traceplot_cal.png')
plt.show()

pm.traceplot(seirdpq_trace_calibration[::plot_step], var_names=('gamma'))
plt.savefig('seirpdq_gamma_traceplot_cal.png')
plt.show()

pm.traceplot(seirdpq_trace_calibration[::plot_step], var_names=('delta'))
plt.savefig('seirpdq_delta_traceplot_cal.png')
plt.show()

# pm.traceplot(seirdpq_trace_calibration[::plot_step], var_names=('omega'))
# plt.savefig('seirpdq_omega_traceplot_cal.png')
# plt.show()

In [ ]:
pm.plot_posterior(seirdpq_trace_calibration[::plot_step], var_names=('beta'), kind='hist', round_to=5)
plt.savefig('seirpdq_beta_posterior_cal.png')
plt.show()

pm.plot_posterior(seirdpq_trace_calibration[::plot_step], var_names=('gamma'), kind='hist', round_to=5)
plt.savefig('seirpdq_gamma_posterior_cal.png')
plt.show()

pm.plot_posterior(seirdpq_trace_calibration[::plot_step], var_names=('delta'), kind='hist', round_to=5)
plt.savefig('seirpdq_delta_posterior_cal.png')
plt.show()

# pm.plot_posterior(seirdpq_trace_calibration[::plot_step], var_names=('omega'), kind='hist', round_to=3)
# plt.savefig('seirpdq_omega_posterior_cal.png')
# plt.show()

In [ ]:
percentile_cut = 5

y_min = target_population * np.percentile(seirdpq_trace_calibration['seirpdq_model'], percentile_cut, axis=0)
y_max = target_population * np.percentile(seirdpq_trace_calibration['seirpdq_model'], 100 - percentile_cut, axis=0)
y_fit = target_population * np.percentile(seirdpq_trace_calibration['seirpdq_model'], 50, axis=0)

In [ ]:
variance_pop = population_uncertain_variance * population_uncertain_variance
sd_pop = np.sqrt(variance_pop)

sd_pop

In [ ]:
plt.figure(figsize=(9, 7))

plt.plot(data_time, y_fit[0], 'b', label='Diagnosed (SEIRPD-Q)', marker='D', linestyle="-", markersize=10)
plt.fill_between(data_time, y_min[0], y_max[0], color='b', alpha=0.2)

plt.plot(data_time, y_fit[1], 'r', label='Deaths (SEIRPD-Q)', marker='v', linestyle="-", markersize=10)
plt.fill_between(data_time, y_min[1], y_max[1], color='r', alpha=0.2)

plt.errorbar(data_time, infected_individuals * target_population, yerr=sd_pop, label='Observed infected', linestyle='None', marker='s', markersize=10)

plt.errorbar(data_time, dead_individuals * target_population, yerr=sd_pop, label='Recorded deaths', marker='v', linestyle="None", markersize=10)

plt.xlabel('Time (days)')
plt.ylabel('Population')
plt.legend()
plt.grid()

plt.tight_layout()

plt.savefig('seirpdq_calibration_bayes.png')
plt.show()